## Import modules and network graph

In [1]:
## all internal libraries
#import pathlib
#import time
#import datetime
#import io
#import urllib
#import tempfile
#
## dependencies
#import shapely.geometry
#import networkx as nx
#import simpy
#import numpy as np
#import pandas as pd
#import geopandas as gpd
#import matplotlib.pyplot as plt
#
## our software
#import opentnsim.core as TNcore
#import openclsim.model as CLmodel
#import openclsim.core as CLcore
#
#%matplotlib inline

In [11]:
import dtv_backend.dtv_backend as backend
import shapely.geometry
import pandas as pd

# our software
import opentnsim.core as TNcore
import openclsim.model as CLmodel
import openclsim.core as CLcore

## Initialize environment with network

In [2]:
# set up environment
env = backend.provide_environment()

# load the network
backend.load_DTV_network_to_env(env)



Defining simulation environment
Starting (down)loading the network
Network succesfully added to simulation


## Define some locations to use

In [3]:
# some interesting locations
locations = {
    'Transferium Maasvlakte': shapely.geometry.Point(4.087406, 51.936737),
    'Neusse': shapely.geometry.Point(6.708892, 51.215737),
    'Basel': shapely.geometry.Point(7.640572, 47.555449),
    'Nijmegen': shapely.geometry.Point(5.8161152, 51.8570535),
    'Waal at St. Andries': shapely.geometry.Point(5.350339, 51.803965)
}

## Define sites 

In [4]:
# TODO: would be nice to define a function that provides the site object for a given point
# automatic ID
# site info (name, capacity, level) should be taken from input, but initialize as e.g. capacity = inf
Site = type(
    "Site",
    (
        CLcore.Identifiable,
        CLcore.Log,
        CLcore.Locatable,
        CLcore.HasContainer,
        CLcore.HasResource,
    ),
    {},
)

# the origin
origin_node = backend.find_closest_node(env.network, locations['Transferium Maasvlakte'])[0]
data_origin_site = {
    "env": env,
    "name": "Origin",
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5b",
    "geometry": env.network.nodes[origin_node]['geometry'],
    "capacity": 10000,
    "level": 10000,
}
origin = Site(**data_origin_site)

# the destinations
destination_node = backend.find_closest_node(env.network, locations['Basel'])[0]
data_destination_site = {
    "env": env,
    "name": "Destination",
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5b",
    "geometry": env.network.nodes[destination_node]['geometry'],
    "capacity": 10000,
    "level": 0,
}
destination = Site(**data_destination_site)


## Create a vessel

In [5]:
# class Metadata:
#    """Something that has a name and id
#    name: a name
#    id: a unique id generated with uuid"""
#
#    def __init__(self, *args, **kwargs):
#        super().__init__()
#        self.metadata = kwargs
#        self.metadata['args'] = args

# the TransportProcessingResource is a mix of openCLSim and openTNSim
# the transport activity (loading, unloading etc) is managed by openCLSim
# the sailing activity and the interaction with the Nt
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        CLcore.Identifiable,
        CLcore.Log,
        CLcore.ContainerDependentMovable,
        #TNcore.ContainerDependentMovable
        TNcore.VesselProperties,
        #TNcore.Routeable,
        CLcore.HasResource,
        CLcore.Processor,
        CLcore.LoadingFunction,
        CLcore.UnloadingFunction,
        TNcore.ExtraMetadata
    ),
    {},
)

def compute_v_provider(v_empty, v_full):
    return lambda x: 10

data_110 = {
    "env": env,
    "name": "NPRC_110",
    "id": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",
    "geometry": env.network.nodes[origin_node]['geometry'],
    "loading_rate": 1000,
    "unloading_rate": 1000,
    "capacity": 3000,
    "compute_v": compute_v_provider(5, 4.5),
    "route": None,
    'vessel_type': 'M8',
    'installed_power': 1000,
    'width': 10, 
    'length': 110, 
    # height above water level
    'height_empty': 8, 
    'height_full': 4, 
    'draught_empty': 2, 
    'draught_full': 6
}

NPRC_110 = TransportProcessingResource(**data_110)

## Functionality to create a single run process but with the moving functionalities of openTNsim

In [6]:
from openclsim.model.move_activity import MoveActivity
from openclsim.model.sequential_activity import SequentialActivity
from openclsim.model.shift_amount_activity import ShiftAmountActivity
from openclsim.model.while_activity import WhileActivity

In [7]:
# once this works it should be moved to the backend, and possibly adapted
def single_run_process(
    env,
    registry,
    name,
    origin,
    destination,
    mover,
    loader,
    unloader,
    start_event=None,
    stop_event=[],
    requested_resources={},
    postpone_start=False,
):
    """Single run activity for the simulation."""
    if stop_event == []:
        stop_event = [
            {
                "or": [
                    {"type": "container", "concept": origin, "state": "empty"},
                    {"type": "container", "concept": destination, "state": "full"},
                ]
            }
        ]
    
    # single run means that we move to origin, load, move to destination, unload
    single_run = [
        MoveActivity(
            env=env,
            registry=registry,
            requested_resources=requested_resources,
            postpone_start=True,
            name=f"{name} sailing empty",
            mover=mover,
            destination=origin,
        ),
        ShiftAmountActivity(
            env=env,
            registry=registry,
            requested_resources=requested_resources,
            postpone_start=True,
            phase="loading",
            name=f"{name} loading",
            processor=loader,
            origin=origin,
            destination=mover,
        ),
        MoveActivity(
            env=env,
            registry=registry,
            requested_resources=requested_resources,
            postpone_start=True,
            name=f"{name} sailing filled",
            mover=mover,
            destination=destination,
        ),
        ShiftAmountActivity(
            env=env,
            registry=registry,
            requested_resources=requested_resources,
            phase="unloading",
            name=f"{name} unloading",
            postpone_start=True,
            processor=unloader,
            origin=mover,
            destination=destination,
        ),
    ]

    # 
    activity = SequentialActivity(
        env=env,
        name=f"{name} sequence",
        registry=registry,
        sub_processes=single_run,
        postpone_start=True,
    )
    
    # 
    while_activity = WhileActivity(
        env=env,
        name=name,
        registry=registry,
        sub_process=activity,
        condition_event=stop_event,
        start_event=start_event,
        postpone_start=postpone_start,
    )

    return single_run, activity, while_activity

## Define a simple openCLSim transport activity from 'from_site' to 'to_site'

In [8]:
single_run, activity, while_activity  = single_run_process(
    name="single_run",
    registry={},
    env=env,
    origin=origin,
    destination=destination,
    mover=NPRC_110,
    loader=NPRC_110,
    unloader=NPRC_110
)

In [9]:
env.run()

In [12]:
pd.DataFrame(NPRC_110.log)

,Message,Timestamp,Value,Geometry,ActivityID,ActivityState
0,move activity single_run sailing empty of NPRC...,2020-11-20 00:00:00.000000,0.0,POINT (4.08441957198572 51.9376698026918),7de5d328-c6f0-4d05-bea6-c274349e3435,START
1,move activity single_run sailing empty of NPRC...,2020-11-20 00:00:00.000000,0.0,POINT (4.08441957198572 51.9376698026918),7de5d328-c6f0-4d05-bea6-c274349e3435,STOP
2,Shift amount activity single_run loading trans...,2020-11-20 00:00:00.000000,3000.0,POINT (4.08441957198572 51.9376698026918),8ad6e5b0-92a7-41d8-8707-63ced04a87fe,START
3,Shift amount activity single_run loading trans...,2020-11-20 00:00:00.000000,3000.0,POINT (4.08441957198572 51.9376698026918),8ad6e5b0-92a7-41d8-8707-63ced04a87fe,START
4,Shift amount activity single_run loading trans...,2020-11-20 00:00:03.000000,3000.0,POINT (4.08441957198572 51.9376698026918),8ad6e5b0-92a7-41d8-8707-63ced04a87fe,STOP
5,Shift amount activity single_run loading trans...,2020-11-20 00:00:03.000000,3000.0,POINT (4.08441957198572 51.9376698026918),8ad6e5b0-92a7-41d8-8707-63ced04a87fe,STOP
6,move activity single_run sailing filled of NPR...,2020-11-20 00:00:03.000000,3000.0,POINT (4.08441957198572 51.9376698026918),1b9c9045-d4dc-4cc3-bdd9-3e77fed7d908,START
7,move activity single_run sailing filled of NPR...,2020-11-20 15:11:09.278951,3000.0,POINT (7.584361254015838 47.57540886152157),1b9c9045-d4dc-4cc3-bdd9-3e77fed7d908,STOP
8,Shift amount activity single_run unloading tra...,2020-11-20 15:11:09.278951,3000.0,POINT (7.584361254015838 47.57540886152157),269ae92e-cf3e-43e8-bf5c-9166c9da663f,START
9,Shift amount activity single_run unloading tra...,2020-11-20 15:11:09.278951,3000.0,POINT (7.584361254015838 47.57540886152157),269ae92e-cf3e-43e8-bf5c-9166c9da663f,START


In [13]:
pd.DataFrame(NPRC_110.log)['Message']

0     move activity single_run sailing empty of NPRC...
1     move activity single_run sailing empty of NPRC...
2     Shift amount activity single_run loading trans...
3     Shift amount activity single_run loading trans...
4     Shift amount activity single_run loading trans...
5     Shift amount activity single_run loading trans...
6     move activity single_run sailing filled of NPR...
7     move activity single_run sailing filled of NPR...
8     Shift amount activity single_run unloading tra...
9     Shift amount activity single_run unloading tra...
10    Shift amount activity single_run unloading tra...
11    Shift amount activity single_run unloading tra...
12    move activity single_run sailing empty of NPRC...
13    move activity single_run sailing empty of NPRC...
14    Shift amount activity single_run loading trans...
15    Shift amount activity single_run loading trans...
16    Shift amount activity single_run loading trans...
17    Shift amount activity single_run loading t

## old code

In [ ]:
# stop before execuring old code
raise

In [ ]:
# Find basic route
start_point = locations_nodes['Transferium Maasvlakte']
end_point = locations_nodes['Nijmegen']
path = nx.dijkstra_path(env.FG, start_point, end_point, weight='Length')

# blockin at...
blocking = locations_nodes['Waal at St. Andries']

# assert that the blocking is in the route
print(f'The blocking is in the path: {blocking in path}')
assert blocking in path, 'Blocking should be in the shortest path'

# quick check on the graph: do the edges have resources?
edge = list(env.FG.edges(blocking))[0]
print(f'Edges have resources? {"Resources" in env.FG.edges[edge].keys()}')

In [ ]:
# get all edges connecting to St. Andries
blocked_edges = []
for edge in env.FG.edges():
    if blocking in edge:
        blocked_edges.append(edge)

# set a trigger (node) for the edge to be blocked 2/4 of the way towards the blocked node
index = int(path.index(blocking)*(2/4))
trigger_node = path[index]

In [ ]:
# Make a transport resource class out of mix-ins
TransportResource = type('TransportResource', 
                         (opentnsim.core.Identifiable, opentnsim.core.ContainerDependentMovable, 
                          opentnsim.core.HasResource, opentnsim.core.Routeable), {})

# For testing purposes we only want v to be 1, whether empty or loaded
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

# Define vessel data, but do not yet link to env or route
data_vessel = {"env": None,
               "name": "Vessel number 1",
               "route": None,
               "geometry": G.nodes[start_point]['geometry'],  # lon, lat
               "capacity": 1_000,
               "compute_v": compute_v_provider(v_empty=1, v_full=1)}


In [ ]:
# create the transport processing resource
vessel1 = TransportResource(**data_vessel)

# Add the vessel to the environment, and define the start and end point
vessel1.env = env
vessel1.start_loc = start_point
vessel1.end_loc = end_point

# also define its current location as the start point for now
vessel1.current_loc = start_point

# add the route
vessel1.route = path

# check route
#vessel1.route

In [ ]:
# create the transport processing resource
vessel2 = TransportResource(**data_vessel)

# Add the vessel to the environment, and define the start and end point
vessel2.env = env
vessel2.start_loc = start_point
vessel2.end_loc = end_point

# also define its current location as the start point for now
vessel2.current_loc = start_point

#--> seems as though this path is sailed even though it is updated on the while loop of the start function

# add the route (is it needed to add it as an empty list?)
#vessel2.route = []

# check route
#vessel2.route

In [ ]:
# simply move a vessel accross a predefined route
def start(env, vessel):
    yield from vessel.move()
    

In [ ]:
# function to sail vessel with route decision
def start_dynamic(env, vessel, blocked_edges, trigger_node):
    """ Have a vessel sail from start_point to end_point, while after every
    egde passed checking if the current route is still available"""
    # while vessel's current location is not end location
    while vessel.current_loc != vessel.end_loc:
        # activate blockage if triggered
        if vessel.current_loc == trigger_node:
            for blocked_edge in blocked_edges:
                env.FG.edges[blocked_edge]['Length'] = 999999
        # determine the shortest path from the current to the end location given the current conditions
        shortest_path = nx.dijkstra_path(env.FG, vessel.current_loc, vessel.end_loc, weight='Length')
        # make sure the vessel passes only the first edge in the route
        vessel.route = [shortest_path[0], shortest_path[1]]
        # make sure the current location is now updated
        vessel.current_loc = shortest_path[1]
        yield from vessel.move()
    

In [ ]:
# add the processes and run the environments
env.process(start(env, vessel1))
env.process(start_dynamic(env, vessel2, blocked_edges, trigger_node))
env.run()

In [ ]:
# check the log of vessel 1
df1 = pd.DataFrame.from_dict(vessel1.log)
df1.head(4).append(df1.tail(4))

In [ ]:
# check the log of vessel 2
df2 = pd.DataFrame.from_dict(vessel2.log)
df2.head(4).append(df2.tail(4))

In [ ]:
# create figure
fix, ax = plt.subplots()

# get log and plot track of vessel 1
df1['x'] = df1.Geometry.apply(lambda geom: geom.x)
df1['y'] = df1.Geometry.apply(lambda geom: geom.y)
df1.plot('x', 'y', label='vessel1', ax=ax)

# get log and plot track of vessel 2
df2['x'] = df2.Geometry.apply(lambda geom: geom.x)
df2['y'] = df2.Geometry.apply(lambda geom: geom.y)
df2.plot('x', 'y', label='vessel2', ax=ax)

In [ ]:
gdf1 = gpd.GeoDataFrame(df1.rename(columns={'Geometry':'geometry'}), crs="EPSG:4326")
gdf2 = gpd.GeoDataFrame(df2.rename(columns={'Geometry':'geometry'}), crs="EPSG:4326")

In [ ]:
import folium
import shapely
m = folium.Map(location=[52, 5], zoom_start=9)
col1 = '#ff0000'
col2 = '#3785b8'

style1 = {'fillColor': col1, 'color': col1, 'line_opacity': 0.2}
style2 = {'fillColor': col2, 'color': col2, 'opacity': 0.8}

folium.GeoJson(shapely.geometry.LineString(gdf1.geometry), style_function=lambda x:style1).add_to(m)
folium.GeoJson(shapely.geometry.LineString(gdf2.geometry), style_function=lambda x:style2).add_to(m)

folium.Marker([G.nodes[blocking]['Y'],G.nodes[blocking]['X']],
          icon = folium.Icon(icon='times', color='red', prefix='fa'),
          ).add_to(m)

m